### Archived pages conversion

In [8]:
from bs4 import BeautifulSoup, SoupStrainer
import os
import requests


In [52]:
script_path = os.getcwd()
data_path = os.path.join(script_path,'data')
intranet_path = os.path.join(data_path,'intranet')
ppl_path = os.path.join(data_path,'people')
start_key = '<div class="et_pb_text_inner">'

In [6]:
r = requests.get(base_url)
rhtml = BeautifulSoup(r.text, 'html.parser')

In [59]:
## Get url links from base page
def clean_links(base_type,base_url):
    r = requests.get(base_url)
    linklist = []
    for link in BeautifulSoup(r.text, 'html.parser', parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            linklist.append(link['href'])
    tmp_links = [x for x in linklist if base_type in x]
    usable_links = [x.replace("http://","https://").replace("https://","https://archive.") for x in tmp_links]
    return usable_links

## Test modules
base_dict = {"intranet":"https://archive.sulab.org/intranet/","author":"https://archive.sulab.org/the-team/"}
base_type = list(base_dict.keys())[0]
base_url = base_dict[base_type]
usable_links = clean_links(base_type,base_url)
print(usable_links)

['https://archive.sulab.org/intranet/su-lab-member-welcome-page/', 'https://archive.sulab.org/intranet/preparing-for-your-move-to-san-diego/', 'https://archive.sulab.org/intranet/lunch-options/', 'https://archive.sulab.org/intranet/group-meeting-schedule/', 'https://archive.sulab.org/intranet/interviewing-notes/', 'https://archive.sulab.org/intranet/scripps-research-travel-reimbursement-policies/', 'https://archive.sulab.org/intranet/conferences-and-training', 'https://archive.sulab.org/intranet/poster-printing/', 'https://archive.sulab.org/intranet/group-committees/', 'https://archive.sulab.org/intranet/titles-and-honorifics/', 'https://archive.sulab.org/intranet/names-and-pronouns', 'https://archive.sulab.org/intranet/organizing-science-projects/', 'https://archive.sulab.org/intranet/coding-standards/', 'https://archive.sulab.org/intranet/giving-great-talks', 'https://archive.sulab.org/intranet/using-amazon-web-services/', 'https://archive.sulab.org/intranet/safety-and-security/']


In [53]:
## parse and save page content as a markdown file

def parse_url_name(url_link):
    url_name = url_link.replace("https://archive.sulab.org/intranet/","").replace("/","")
    url_name = url_link.replace("https://archive.sulab.org/author/","").replace("/","")
    return url_name

def get_content(url_link):
    r = requests.get(url_link)
    r_page = BeautifulSoup(r.text, 'html.parser')
    r_content = r_page.select('div.et_pb_text_inner')[0]
    return r_content
    
def convert_text(r_content):
    headtext = r_content.h1.get_text()
    bodytext = r_content.get_text()
    lesshead = bodytext.replace(headtext,"")
    clean_text = "---\ntitle: " + headtext + '\n---\n{% include breadcrumbs.html %}\n\n' + lesshead
    return clean_text

## Test modules
test_link = usable_links[0]
print(test_link)
file_name = parse_url_name(test_link)
test_content = get_content(test_link)
clean_test = convert_text(test_content)
print(file_name)
print(clean_test[0:100])

https://archive.sulab.org/intranet/su-lab-member-welcome-page/
https:archive.sulab.orgintranetsu-lab-member-welcome-page
---
title: Welcome to the Su Lab!
---
{% include breadcrumbs.html %}


We’re excited to have you joi


In [51]:
def make_files(data_path,usable_links):
    for eachlink in usable_links:
        try:
            each_content = get_content(eachlink)
            file_name = parse_url_name(eachlink)
            clean_text = convert_text(each_content)
            with open(os.path.join(data_path,f"{file_name}.md"),'w') as outwrite:
                outwrite.write(clean_text)
        except:
            print("failed: ", eachlink)

make_files(intranet_path,usable_links)

failed:  https://archive.sulab.org/intranet/group-meeting-schedule/
